In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType

In [2]:
spark = SparkSession.builder.appName("final_task").config('spark.master', 'local[*]').getOrCreate()

24/07/24 13:35:53 WARN Utils: Your hostname, themjdex-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/07/24 13:35:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 13:35:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('data/web_server_logs.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [5]:
# конвертируем timestamp в тип date
df=df.withColumn('timestamp', df['timestamp'].cast(DateType()))
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [6]:
df.createOrReplaceTempView('traffic')

In [7]:
# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP
print('Top 10 active IP adresses:')
spark.sql('''
          SELECT ip, COUNT(url) AS request_count 
          FROM traffic 
          GROUP BY ip 
          ORDER BY request_count DESC 
          LIMIT 10
          ''').show()

# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода
print('Request count by HTTP method:')
spark.sql('''
          SELECT method, COUNT(method) AS method_count 
          FROM traffic 
          GROUP BY method 
          ''').show()

# Профильтруйте и посчитайте количество запросов с кодом ответа 404
count_404 = spark.sql('''
          SELECT COUNT(response_code)
          FROM traffic 
          WHERE response_code = 404 
          ''').collect()
print(f'Number of 404 response codes: {count_404[0][0]}\n')

# Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате
print('Total response size by day:')
spark.sql('''
          SELECT timestamp AS date, SUM(response_size) AS total_response_size 
          FROM traffic 
          GROUP BY date
          ORDER BY date
          ''').show()

Top 10 active IP adresses:


+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
| 207.90.62.173|            2|
| 203.50.53.156|            2|
|188.56.188.241|            2|
|104.225.103.17|            2|
| 30.86.184.103|            1|
|131.254.187.25|            1|
|181.96.201.208|            1|
| 199.185.71.89|            1|
|  64.18.38.139|            1|
|156.25.119.206|            1|
+--------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24862|
|DELETE|       25222|
|   PUT|       24868|
|   GET|       25048|
+------+------------+

Number of 404 response codes: 25164

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            2311466|
|2024-01-02|            2391624|
|2024-01-03|            2239093|
|2024-01-04|            2342243|
|2024-01-05|            2440897|
|2024-01-06|            25397

In [8]:
spark.stop()